In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
#| default_exp sim

In [ ]:
#| export

device='cpu'
import torch
import numpy as np
import random
import matplotlib.pyplot as plt

import torch
import numpy as np
import random

def set_seed(seed):
    if seed is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
### BREEDING SIMULATOR
class Genome:
    def __init__(self, n_chr, n_loci, seed=None):
        self.ploidy = 2
        self.n_chr = n_chr
        self.n_loci = n_loci
        self.shape = (self.ploidy, self.n_chr, self.n_loci)


class Population:
    def __init__(self, genome, haplotypes, device=device):
        self.genome = genome
        self.device = device
        self.phenotypes = None
        self.bvs = None
        self.haplotypes = haplotypes.to(device)
        self.dosages = haplotypes.sum(dim=1).float().to(device)
        self.size = haplotypes.shape[0]

class Trait:
    def __init__(self, genome, founder_population, target_mean, target_variance, device=device, seed=None):
        set_seed(seed)
        self.target_mean = target_mean
        self.target_variance = target_variance
        self.device = device
        
        random_effects = torch.randn(genome.n_chr, genome.n_loci, device=self.device)
        random_effects -= random_effects.mean()
        
        founder_scores = torch.einsum('kl,hkl->h', random_effects, founder_population.dosages).to(device)
        founder_mean, founder_var = founder_scores.mean(), founder_scores.var()
        
        scaling_factors = torch.sqrt(self.target_variance / founder_var)
        self.scaling_factors = scaling_factors
        random_effects *= scaling_factors
        self.effects = random_effects
        self.intercept = founder_mean - target_mean

class SimParams:
    def __init__(self, founder_pop, config):
        self.G = Genome(config['n_chr'], config['n_loci'])
        self.T = Trait(self.G, founder_pop, config['target_mean'], config['target_variance'], seed=config['seed'])
        self.h2 = config['h2']
        self.reps = config['reps']
        self.pop_size = config['pop_size']
        self.max_generations = config['start_gen']  # Initialize with start_gen
        self.end_gen = config['end_gen']  # Store end_gen for reference
        self.founder_pop = founder_pop
        self.seed = config['seed']
        if self.seed is not None:
            set_seed(self.seed)

    def update_max_generations(self, new_max_gen):
        self.max_generations = min(new_max_gen, self.end_gen)

def scale_values(x, from_range=(-1, 1), to_range=(0.05, 0.95)):
    # Ensure input is a PyTorch tensor
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
    
    # Unpack the ranges
    from_min, from_max = from_range
    to_min, to_max = to_range
    
    # Scale the values
    scaled = (x - from_min) / (from_max - from_min) * (to_max - to_min) + to_min
    
    return scaled


## ACTIONS

def calculate_breeding_value(population, trait, device=device):
    return torch.einsum('hjk,jk->h', population.dosages, trait.effects).to(device)

def truncation_selection(population, trait, top_percent):
    return torch.topk(population.phenotypes, top_percent).indices.to(device)



def bv(P,T):
    P.breeding_values = calculate_breeding_value(P.dosages,T.effects)

def create_pop(G, haplotypes):
    return Population(G, haplotypes=haplotypes)

def create_random_pop(G, pop_size):
    return torch.randint(0, 2, (pop_size, *G.shape), device=device)

def update_pop(population, haplotype_pop_tensor):
    population.haplotypes = haplotype_pop_tensor
    population.dosages = haplotype_pop_tensor.sum(dim=1).float()
    return population





# meiosis
def recombine(parent_haplo_tensor, recombination_rate=0.1):
    num_individuals, ploidy, num_chromosomes, num_loci = parent_haplo_tensor.shape
    # Generate crossover masks
    maternal, paternal = parent_haplo_tensor[:,0,:,:],parent_haplo_tensor[:,1,:,:],
    crossovers = torch.bernoulli(torch.full((num_individuals, num_chromosomes, num_loci), recombination_rate, device=device))
#     crossovers = torch.rand((num_individuals, num_chromosomes, num_loci), device=device) < recombination_rate
    progeny = maternal * torch.logical_not(crossovers) + paternal * crossovers
    return progeny

def breed(mother_tensor, father_tensor, recombination_rate=0.1):
    eggs = recombine(mother_tensor,recombination_rate)
    pollens = recombine(father_tensor,recombination_rate)
    return torch.stack((eggs,pollens), dim=1)



def create_progeny(mother_gametes, father_gametes, reps=1, device=device):
    # Repeat gametes along a new dimension for replications
    mother_gametes = mother_gametes.unsqueeze(1).repeat(1, reps, 1, 1, 1)
    father_gametes = father_gametes.unsqueeze(1).repeat(1, reps, 1, 1, 1)

    # Shuffle indices for all replications at once
    shuffled_mother_indices = torch.randperm(mother_gametes.shape[0], device=device)
    shuffled_father_indices = torch.randperm(father_gametes.shape[0], device=device)
    shuffled_mother_indices = shuffled_mother_indices.unsqueeze(1).repeat(1, reps)
    shuffled_father_indices = shuffled_father_indices.unsqueeze(1).repeat(1, reps)

    # Gather shuffled gametes and stack
    mother_gametes = torch.gather(mother_gametes, 0, shuffled_mother_indices.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(-1, -1, ploidy, num_chromosomes, num_loci))
    father_gametes = torch.gather(father_gametes, 0, shuffled_father_indices.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(-1, -1, ploidy, num_chromosomes, num_loci))
    progeny_haplotypes = torch.stack((mother_gametes, father_gametes), dim=2)
    return progeny_haplotypes.reshape(-1, 2, num_chromosomes, num_loci)


def random_crosses(parent_population, total_crosses, device=device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    
    num_parents = parent_population.shape[0]
    ploidy, num_chromosomes, num_loci = parent_population.shape[1:]
    
    # Randomly select parents for each cross
    parent_indices = torch.randint(0, num_parents, (total_crosses, 2), device=device)
    
    # Select the parent haplotypes
    mothers = parent_population[parent_indices[:, 0]]
    fathers = parent_population[parent_indices[:, 1]]
    
    # Perform recombination for both parents
    mother_gametes = recombine(mothers)
    father_gametes = recombine(fathers)
    
    # Combine gametes to create progeny
    progeny = torch.stack((mother_gametes, father_gametes), dim=1)
    
    return progeny

def phenotype(population, trait, h2):
    breeding_values = calculate_breeding_value(population, trait)
    population.breeding_values = breeding_values
    population.genetic_var = breeding_values.var()
    if breeding_values.var() == 0:
        environmental_variance = 0
    else:
        environmental_variance = (1 - h2) / h2 * breeding_values.var()

    # Check if environmental_variance is zero before applying torch.sqrt and .clone()
    if environmental_variance == 0:
        environmental_noise = torch.zeros(breeding_values.shape, device=device)
    else:
        environmental_noise = torch.randn(breeding_values.shape, device=device) * torch.sqrt(environmental_variance).detach()
    population.phenotypes = breeding_values + environmental_noise


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()